In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split

eurosatNTData = torchvision.datasets.EuroSAT(root = './data', download = True, transform = None)
device =  "cuda" if torch.cuda.is_available() else "cpu"
print(device)
useAugmentation = True

cuda


In [2]:
def GetLoaders(isAugmented = False, batchSize = 32):

  indices = list(range(len(eurosatNTData)))
  labels = eurosatNTData.targets

  trainIdx, testIdx = train_test_split(
      indices,
      test_size=0.1,
      random_state=42,
      stratify=labels
  )

  noAUGTransform = v2.Compose([
      v2.ToTensor(),
      v2.Normalize(mean = [0.5], std = [0.5]),
  ])

  AUGTransform = v2.Compose([
    v2.RandomPerspective(distortion_scale=0.2, p=0.5),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    v2.RandomApply([v2.ColorJitter(brightness=0.2, contrast=0.2)],p=0.8),
    v2.RandomApply([v2.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 1))],p=0.3),
    v2.ToTensor(),
    v2.Normalize(mean = [0.5], std = [0.5])
])


  if(isAugmented):
    notAugmentedTrain = Subset(torchvision.datasets.EuroSAT(root="./data", transform=noAUGTransform),trainIdx)
    augmentedTrain = Subset(torchvision.datasets.EuroSAT(root="./data", transform=AUGTransform),trainIdx)
    trainDataset = torch.utils.data.ConcatDataset([augmentedTrain,notAugmentedTrain])
  else:
    trainDataset = Subset(torchvision.datasets.EuroSAT(root="./data", transform=noAUGTransform),trainIdx)

  testDataset = Subset(torchvision.datasets.EuroSAT(root="./data", transform=noAUGTransform),testIdx)
  trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size = batchSize, shuffle = True, num_workers = 10, pin_memory=True)
  testLoader = torch.utils.data.DataLoader(testDataset, batch_size = batchSize, shuffle = False)
  print(len(trainDataset))
  return trainLoader, testLoader


In [3]:
trainLoader, testLoader = GetLoaders(useAugmentation, 64)

48600


C:\Users\Konstantinos\miniconda3\envs\torch-nightly\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [4]:
class CNN(nn.Module):
  def __init__(self, inChannels, numClasses):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = inChannels, out_channels= 16, kernel_size = 3, stride = 1, padding = 1)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
    self.fc1 = nn.Linear(32 * 16 * 16, 128)
    self.fc2 = nn.Linear(128, numClasses)
    self.dropfc1 = nn.Dropout(p=0.2)
    # self.dropconv2 = nn.Dropout2d(p=0.1)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.view(-1, 32 * 16 * 16)
    x = F.relu(self.dropfc1(self.fc1(x)))
    x = self.fc2(x)
    return x

model = CNN(inChannels = 3, numClasses = 10).to(device)

In [5]:
def check_accuracy():
    numCorrect = 0
    numSamples = 0
    model.eval()  

    with torch.no_grad(): 
        for x, y in testLoader:
            
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            
            scores = model(x)
            _, predictions = scores.max(1)  
            numCorrect += (predictions == y).sum() 
            numSamples += predictions.size(0) 
        
        accuracy = float(numCorrect) / float(numSamples) * 100
        print(f"Got {numCorrect}/{numSamples} with accuracy {accuracy:.2f}%")

    model.train()  

check_accuracy()

Got 322/2700 with accuracy 11.93%


In [6]:
#Hyper parameters
learningRate = 1e-3
epochs = 120 if useAugmentation else 80
lossFunction = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learningRate)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=epochs)
losses = []

In [7]:
def train():
  for epoch in range(epochs):
    epochLoss = 0
    numBatches = 0
    for X, y in trainLoader:
      X = X.to(device, non_blocking=True)
      y = y.to(device, non_blocking=True)
      optimizer.zero_grad()
      pred = model(X)
      loss = lossFunction(pred, y)
      loss.backward()
      optimizer.step()
      epochLoss += loss.item()
      numBatches += 1
    scheduler.step()
    avgLoss = epochLoss/numBatches
    losses.append(avgLoss)
    print(f'Epoch {epoch+1}, Loss: {avgLoss:.9f}, Lr: {optimizer.param_groups[0]["lr"]:.6}')

train()

Epoch 1, Loss: 1.091695022, Lr: 0.000999829
Epoch 2, Loss: 0.766815652, Lr: 0.000999315
Epoch 3, Loss: 0.665550540, Lr: 0.000998459
Epoch 4, Loss: 0.589989481, Lr: 0.000997261
Epoch 5, Loss: 0.552853293, Lr: 0.000995722
Epoch 6, Loss: 0.494881696, Lr: 0.000993844
Epoch 7, Loss: 0.461468774, Lr: 0.000991627
Epoch 8, Loss: 0.421436748, Lr: 0.000989074
Epoch 9, Loss: 0.400036345, Lr: 0.000986185
Epoch 10, Loss: 0.364083334, Lr: 0.000982963
Epoch 11, Loss: 0.353041009, Lr: 0.00097941
Epoch 12, Loss: 0.323468184, Lr: 0.000975528
Epoch 13, Loss: 0.312444796, Lr: 0.000971321
Epoch 14, Loss: 0.294227417, Lr: 0.00096679
Epoch 15, Loss: 0.284009359, Lr: 0.00096194
Epoch 16, Loss: 0.271584598, Lr: 0.000956773
Epoch 17, Loss: 0.260123072, Lr: 0.000951293
Epoch 18, Loss: 0.248366171, Lr: 0.000945503
Epoch 19, Loss: 0.248990152, Lr: 0.000939409
Epoch 20, Loss: 0.232897308, Lr: 0.000933013
Epoch 21, Loss: 0.234989608, Lr: 0.00092632
Epoch 22, Loss: 0.220402986, Lr: 0.000919335
Epoch 23, Loss: 0.22229

In [10]:
def check_accuracy():
    numCorrect = 0
    numSamples = 0
    model.eval()  

    with torch.no_grad(): 
        for x, y in testLoader:
            
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            
            scores = model(x)
            _, predictions = scores.max(1)  
            numCorrect += (predictions == y).sum() 
            numSamples += predictions.size(0) 
        
        accuracy = float(numCorrect) / float(numSamples) * 100
        print(f"Got {numCorrect}/{numSamples} with accuracy {accuracy:.2f}%")

    model.train()  

check_accuracy()

Got 2518/2700 with accuracy 93.26%


In [11]:
torch.save(model.state_dict(), "model.pth")